In [1]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy import signal
import time
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
WORKSPACE_PATH = '/home/takeyama/pywork/ipython/2016-06-06/'

mozartの環境設定

In [3]:
#p_path="/home/takeyama/pywork/ipython/2016-05-30/"

In [4]:
#cd ~/Documents/SyncRecord/cleaning-addingLABEL/

workstationの環境設定

In [5]:
cd ~/Documents/AddingLabel+FFTdata/

/home/takeyama/Documents/AddingLabel+FFTdata


In [6]:
data = pd.read_csv('west.csv',encoding="SHIFT-JIS")
data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
data = data[data.Type=='ags']
data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])

# convert numpy.darray
AccX=data.AccX.values*0.0001
AccY=data.AccY.values*0.0001
AccZ=data.AccZ.values*0.0001
GyrX=data.GyrX.values*0.01
GyrY=data.GyrY.values*0.01
GyrZ=data.GyrZ.values*0.01
Time=data.index

RawData = np.vstack( (AccX,AccY,AccZ,GyrX,GyrY,GyrZ,Time))

# regist each raw data 
"""
RawData['AccX'] = AccX
RawData['AccY'] = AccY
RawData['AccZ'] = AccZ
RawData['GyrX'] = GyrX
RawData['GyrY'] = GyrY
RawData['GyrZ'] = GyrZ
RawData['Time'] = Time
"""

"\nRawData['AccX'] = AccX\nRawData['AccY'] = AccY\nRawData['AccZ'] = AccZ\nRawData['GyrX'] = GyrX\nRawData['GyrY'] = GyrY\nRawData['GyrZ'] = GyrZ\nRawData['Time'] = Time\n"

In [26]:
class SensorData:
    
    def __init__(self):
        print "__class__"
        # raw data
        self._RawData={}    
        # fft data
        self._FFTData={}
        # power spectol data
        self._PowerData={}    
        # flag exsist data
        self._Flag_exist_data=False
        
        self._columns=['AccX','AccY','AccZ','GyrX','GyrY','GyrZ']
        self._fft_col=['fft_AccX','fft_AccY','fft_AccZ','fft_GyrX','fft_GyrY','fft_GyrZ']
        self._power_col=['power_AccX','power_AccY','power_AccZ','power_GyrX','power_GyrY','power_GyrZ']
        
    def ImportCSV(self,Sclass,csv_file):
        self.ClassName=Sclass
        self.Flag_exist_data=True
        # design dataframe
        data = pd.read_csv(csv_file,encoding="SHIFT-JIS")
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data = data[data.Type=='ags']
        data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])
        
        # convert numpy.darray
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
        Time=data.index
        
        # regist each raw data 
        self._RawData['AccX'] = AccX
        self._RawData['AccY'] = AccY
        self._RawData['AccZ'] = AccZ
        self._RawData['GyrX'] = GyrX
        self._RawData['GyrY'] = GyrY
        self._RawData['GyrZ'] = GyrZ
        self._RawData['Time'] = Time
    
    def ShowFlagExistData(self):
        return self.Flag_exist_data
    
    def GetColumns(self):
        return self._columns
    
    def GetTime(self):
        return self._RawData['Time']

    def ShowAllDf(self):
        print 'AccX : ';print self._RawData['AccX']
        print 'AccY : ';print self._RawData['AccY']
        print 'AccZ : ';print self._RawData['AccZ']
        print 'GyrX : ';print self._RawData['GyrX']
        print 'GyrY : ';print self._RawData['GyrY']
        print 'GyrZ : ';print self._RawData['GyrZ']
        
    def _Time2Num(self,time):
        return np.where(self._RawData['Time']==np.datetime64(time) )[0][0]
    
    def ShowQuery(self,Sname,rng=[]):
        data = self._RawData[Sname]
        print Sname+':'+str( data[rng[0]:rng[1]])
            
    def _sliding_window(self,Sname,samp, overlap):
        count =0
        s =self._RawData['Time'][0]
        start=self._Time2Num(s)
        g = s+np.timedelta64(samp*10,'ms')
        goal= self._Time2Num(g)
        yield self._RawData[Sname][start:goal]
        
        add=overlap*0.01
        
        while True:
            try:
                count +=1
                s =s+np.timedelta64(samp*10,'ms')
                start=self._Time2Num(s)
                g  =s+np.timedelta64(samp*10,'ms')
                goal= self._Time2Num(g)
                yield self._RawData[Sname][start:goal]
            except StopIteration:
                print '_sliding_window StopIteration'
                break   
            except IndexError:
                print '_sliding_window IndexError'
                break  
                
# Fast Frier transaction            
    def GetFFT(self,Sfft,samp):
        return np.load(p_path+'fft/'+self.ClassName+'_'+Sfft+'_'+str(samp)+'.npz')['arr_0']
    
    def CalcFFT(self,samp,overlap=0.5):
        start = time.time()
        fft_data = np.array([])
        
        for n,f in zip( self._columns,self._fft_col):
            print 'start'+n+'->'+f
            sw = self._sliding_window(n,samp,overlap)
            while True:
                try:
                    d=sw.next()
                    fft_data = np.append(fft_data, fftpack.fft(d)[1:(samp/2)+1] )   # fftの直流成分を除くsample/2の
                except StopIteration:
                    print 'CalcFFTStopIteration'
                    fft_data = fft_data.reshape(len(fft_data)/(samp/2),(samp/2) )
                    self._FFTData[f] = fft_data
                    np.savez(p_path+'fft/'+self.ClassName+'_'+str(f)+'_'+str(samp),self._FFTData[f])
                    break   
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
           

# Spectol Power 
    def GetPower(self,Spower,samp):
        return np.load(p_path+'power/'+self.ClassName+'_'+Spower+'_'+str(samp)+'.npz')['arr_0']
    
    def _power(self,fft_array):
        p=lambda x,y : np.sqrt(x**2+y**2)
        power_array=np.array([])
        for vector in fft_array:
            tmp = p( np.real(vector),np.imag(vector) )
            tmp = tmp/np.sum(tmp)
            power_array = np.append(power_array,tmp)
        return power_array
        
    def CalcPower(self,samp,overlap=0.5):
        start = time.time()
        for fft_name,power_name in zip( self._fft_col, self._power_col):
            print 'start'+fft_name+'->'+power_name
            fft_data = self.GetFFT(fft_name,samp)
            power_data=self._power(fft_data)
            np.savez(p_path+'power/'+self.ClassName+'_'+power_name+'_'+str(samp),power_data)
        elapsed_time = time.time() - start
        print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

In [67]:
'''
csv_file -> ファイル名
mode Round -> 四捨五入
     Roundup -> 切り上げ
     Rounddown -> 切り捨て
'''
def ImportCSV(csv_file,samp,mode='Round'):
        # data dictionary 
        RawData={}   
        
        # design dataframe and import csv
        data = pd.read_csv('west.csv',encoding="SHIFT-JIS")
        data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
        data = data[data.Type=='ags']
        data = pd.pivot_table(data,values=[u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ'],index=[u'Time'])

        # convert numpy.darray
        AccX=data.AccX.values*0.0001
        AccY=data.AccY.values*0.0001
        AccZ=data.AccZ.values*0.0001
        GyrX=data.GyrX.values*0.01
        GyrY=data.GyrY.values*0.01
        GyrZ=data.GyrZ.values*0.01
       
        # regist each raw data 
        RawData['AccX'] = AccX
        RawData['AccY'] = AccY
        RawData['AccZ'] = AccZ
        RawData['GyrX'] = GyrX
        RawData['GyrY'] = GyrY
        RawData['GyrZ'] = GyrZ
        
        # import time by using numpy
         Time=data.index #時間の列だけを抽出       
        
        if mode == 'Roundup':
            func = lambda x: int(x/samp)*samp
        elif mode == 'Rounddown':
            func = lambda x: int(x/samp)*samp
        elif mode == 'Round':
            func = lambda x: int((x+samp/2)/samp)*samp
        #ERROR
        else:
            print 'check mode and inputed word is caused error'
            return -1
        
        output = map(func,Time)
        RawData['Time'] = output
        return RawData
            

IndentationError: unexpected indent (<ipython-input-67-86bb4d6fde98>, line 34)

In [68]:
data1=ImportCSV('left-hand.csv',16)
data2=ImportCSV('left-leg.csv',16)
data3=ImportCSV('right-hand.csv',16)
data4=ImportCSV('right-leg.csv',16)
data5=ImportCSV('west.csv',16)

[65175712, 65175712, 65175728, 65175744, 65175744, 65175760, 65175760, 65175776, 65175792, 65175792, 65175808, 65175824, 65175824, 65175840, 65175840, 65175856, 65175872, 65175872, 65175888, 65175904, 65175904, 65175920, 65175920, 65175936, 65175952, 65175952, 65175968, 65175984, 65175984, 65176000, 65176000, 65176016, 65176032, 65176032, 65176048, 65176064, 65176064, 65176080, 65176080, 65176096, 65176112, 65176112, 65176128, 65176144, 65176144, 65176160, 65176160, 65176176, 65176192, 65176192, 65176208, 65176224, 65176224, 65176240, 65176240, 65176256, 65176272, 65176272, 65176288, 65176304, 65176304, 65176320, 65176320, 65176336, 65176352, 65176352, 65176368, 65176384, 65176384, 65176400, 65176400, 65176416, 65176432, 65176432, 65176448, 65176464, 65176464, 65176480, 65176480, 65176496, 65176512, 65176512, 65176528, 65176544, 65176544, 65176560, 65176560, 65176576, 65176592, 65176592, 65176608, 65176624, 65176624, 65176640, 65176640, 65176656, 65176672, 65176672, 65176688, 65176704,

In [56]:
def CalcStartTime(array):
    MAX = min(array[0])
    
    for i in range(len(array)):
        if MAX < min(array[i]):
            MAX = min(array[i])
     
    return  MAX

In [57]:
def CalcGoalTime(array):
    MIN = max(array[0])
    
    for i in range(len(array)):
        if MIN > max(array[i]):
            MIN = max(array[i])
     
    return  MIN

**2016-06-14**  
もう１度、前処理について考える。特に、入力データの特徴、出力データの仕様を改める。  
*feacher of Precondition*
1. センサデータは{time, accx, accy, accz, gyrx, gyry, gyrz}が揃っている
2. また、センサデータはすべて整数型となっている。
3. nanデータはない

*output Data*
1. accは[0.1mG]から[G]に単位変換、gyrは[0.01dpg]は[dpg]に変換する。
2. timeはサンプリング周期で丸める。
3. Nanデータは存在する。


In [58]:
def NanPating(checkData):
    import time
    start_time = time.time()
    tmp = np.array([])
    
    print 'len( checkData ) = '+str(len( checkData ))
    
    # detection for hidden Nan Data
    diffNum =[]
    diffIndex=[]
    width = len(checkData)
    for i in range(0,width-1):
        if ( checkData[i+1]-checkData[i] )!=10:
            diffNum.append(int(checkData[i+1]-checkData[i]) )
            diffIndex.append(i)
    print 'diffNum='+str(diffNum)+'diffIndex='+str(diffIndex)
    
    # insert NAN data
    StartIndex= 0
    tmp =np.array([])
    for count,l in enumerate(diffIndex):
        tmp = np.append(tmp, checkData[StartIndex:l])
        d = tmp
        print 'tmp + rawdata len='+str(len(tmp))
        for i in range(0,int(diffNum[count]/10) ):
            tmp = np.append(tmp,np.nan)
        print 'tmp + nan len='+str(len(tmp)-len(d))
        StartIndex = l+1
    tmp=np.append(tmp, checkData[StartIndex:])
    print 'nan adding data ='+str( len(tmp) )
    print len(tmp)
    elapsed_time = time.time() -start_time
    print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"
    return tmp

In [59]:
data5['Time']

[65175760,
 65175776,
 65175776,
 65175792,
 65175792,
 65175808,
 65175824,
 65175824,
 65175840,
 65175856,
 65175856,
 65175872,
 65175872,
 65175888,
 65175904,
 65175904,
 65175920,
 65175936,
 65175936,
 65175952,
 65175952,
 65175968,
 65175984,
 65175984,
 65176000,
 65176016,
 65176016,
 65176032,
 65176032,
 65176048,
 65176064,
 65176064,
 65176080,
 65176096,
 65176096,
 65176112,
 65176112,
 65176128,
 65176144,
 65176144,
 65176160,
 65176176,
 65176176,
 65176192,
 65176192,
 65176208,
 65176224,
 65176224,
 65176240,
 65176256,
 65176256,
 65176272,
 65176272,
 65176288,
 65176304,
 65176304,
 65176320,
 65176336,
 65176336,
 65176352,
 65176352,
 65176368,
 65176384,
 65176384,
 65176400,
 65176416,
 65176416,
 65176432,
 65176432,
 65176448,
 65176464,
 65176464,
 65176480,
 65176496,
 65176496,
 65176512,
 65176512,
 65176528,
 65176544,
 65176544,
 65176560,
 65176576,
 65176576,
 65176592,
 65176592,
 65176608,
 65176624,
 65176624,
 65176640,
 65176656,
 65176656,

In [60]:
time5=NanPating()

TypeError: NanPating() takes exactly 1 argument (0 given)

In [61]:
timeArray=[time1,time2,time3,time4,time5]

In [62]:
CalcStartTime(timeArray)

65175887

具体的なnumpyのloadtxt,genfromtxtの違い

np.loadtxt('left-hand.csv',delimiter=',',usecols=(2,3,4))

np.genfromtxt('left-hand.csv',delimiter=',',usecols=(2,3,4))